# 1. Baseline

## Libraries

In [7]:
import os 
from pathlib import Path
from tqdm import tqdm
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import transforms as trans

from data.ms1m import get_train_loader
from data.lfw import LFW

from backbone.arcfacenet import SEResNet_IR
from margin.ArcMarginProduct import ArcMarginProduct

from util.utils import save_checkpoint, test

## Configuration

In [8]:
conf = edict()

conf.train_root = "./dataset/MS1M"
conf.lfw_test_root = "./dataset/lfw_aligned_112"
conf.lfw_file_list = "./dataset/lfw_pair.txt"

conf.mode = "se_ir" # "ir = Resnet Tabanlı"
conf.depth = 50  #50,100,152 #Derinliği artırırsak uzun sürer eğitiim, 100 kullanırsak ir kullanmak daha iyi
conf.margin_type = "ArcFace"
conf.feature_dim = 512 #Genelde 512 kullanılır daha iyi sonuç alınıyor 200 de olabilir 
conf.scale_siz = 32.0
conf.batch_size = 96 #Modele kaçar kaçar vereceğimizi söylüyoruz
conf.lr = 0.01 #Batch size ile lr doğru orantı var, batch size düşürüyorsak lr de düşürmemiz gerekiyor
conf.milestones = [8,10,12]#Eğitim kaybımızı hangi epochlarda kontrol edeceğiz, öğrenme oranını 8,10,12 de düşürüypruz
conf.total_epoch = 14

conf.save_folder = "./saved"
conf.save_dir = os.path.join(conf.save_folder,conf.mode + "_" + str(conf.depth) )#./saved/se_ir_50
conf.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #GPU cuda =0
conf.num_workers = 4
conf.pin_memory = True

In [9]:
os.makedirs(conf.save_dir, exist_ok = True)

## Data Loader

In [11]:
transform = trans.Compose([
    trans.ToTensor(), #RANGE [0,255] -> [0.0, 1.0]
    trans.Normalize(mean=(0.5,0.5,0.5), std = (0.5,0.5,0.5))
])
trainloader, class_num = get_train_loader(conf)

In [12]:
print("number of id ",class_num)

number of id  200


In [13]:
print(trainloader.dataset)

Dataset ImageFolder
    Number of datapoints: 29148
    Root Location: ./dataset/MS1M
    Transforms (if any): Compose(
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                         )
    Target Transforms (if any): None


In [18]:
lfwdataset = LFW(conf.lfw_test_root,conf.lfw_file_list,transform = transform)
lfwloader = torch.utils.data.DataLoader(lfwdataset,batch_size = 128,num_workers = conf.num_workers)

In [19]:
print(conf.device)

cpu


# Model

In [ ]:
net = SEResNet_IR(conf.depth,feature_dim = conf.feature_dim,mode = conf.mode)to(conf.device)

## Train